In [147]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from datetime import datetime

In [148]:
%run Data_Cleaning_Utils.ipynb

In [149]:
data_path = './PM2.5/'
year_scope = [2017, 2018, 2019, 2020, 2021, 2022]
os.listdir(data_path)
data_name_lst = [name for name in os.listdir(data_path) if name.startswith('PM2.5')]

In [5]:
selected_data_name = []
for i in year_scope:
    for j in data_name_lst:
        if str(i) in j:
            selected_data_name.append(j)

In [6]:
selected_data_name

['PM2.5_2017_.xlsx',
 'PM2.5_2018_.xlsx',
 'PM2.5_2019_.xlsx',
 'PM2.5_2020_.xlsx',
 'PM2.5_2021_.xlsx',
 'PM2.5_2022_.xlsx']

## Working Space

### Observing on year data

In [7]:
locate_lst, ref_station = made_locate_station(selected_data_name[-1], selected_data_name, data_path)

In [8]:
df = pd.DataFrame(locate_lst, columns=ref_station)
df_year = pd.DataFrame(year_scope, columns=['years'])
df = df_year.join(df)
df = df.loc[:,~df.columns.duplicated()].copy()
df

,years,02T,05T,10T,11T,12T,59T,61T,03T,50T,...,42T,43T,44T,62T,63T,78T,80T,89T,93T,103T
0,2017,0,1,0,0,0,1,1,0,1,...,0,0,1,1,1,0,1,0,0,0
1,2018,0,1,1,1,0,1,1,1,1,...,1,0,1,1,1,0,1,0,0,0
2,2019,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,0,0,0
3,2020,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,0,0,0
4,2021,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,0
5,2022,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


### Observing on the provide

In [9]:
provide_station_df = made_provide_station(selected_data_name, data_path)

In [13]:
provide_station_df

,station_id,station_name,label
0,05T,กทม.,0
1,59T,กทม.,0
2,61T,กทม.,0
3,50T,กทม.,0
4,53T,กทม.,0
...,...,...,...
82,95T,จ.พิจิตร,24
83,96T,จ.สุโขทัย,44
84,97T,จ.เพชรบูรณ์,52
85,98T,จ.อุทัยธานี,48


### Observing on the area

<img src="Thailand_section_meteorology.png" alt="Alternative text" />

In [134]:
NE = ' อำนาจเจริญ, บึงกาฬ, บุรีรัมย์, ชัยภูมิ, กาฬสินธุ์, ขอนแก่น, เลย, มหาสารคาม, มุกดาหาร, นครพนม, นครราชสีมา, หนองบัวลำภู, หนองคาย, ร้อยเอ็ด, สกลนคร, ศรีสะเกษ, สุรินทร์, อุบลราชธานี, อุดรธานี, ยโสธร'
N = ' เชียงใหม่, เชียงราย, ลำปาง, ลำพูน, แม่ฮ่องสอน, น่าน, พะเยา, แพร่, อุตรดิตถ์, ตาก, สุโขทัย, พิษณุโลก, พิจิตร, กำแพงเพชร, เพชรบูรณ์'
C = ' นครสวรรค์, อุทัยธานี, อ่างทอง, ชัยนาท, พระนครศรีอยุธยา, กทม., ลพบุรี, นครปฐม, นนทบุรี, ปทุมธานี, สมุทรปราการ, สมุทรสาคร, สมุทรสงคราม, สระบุรี, สิงห์บุรี, สุพรรณบุรี, กาญจนบุรี, ราชบุรี'
E = ' ฉะเชิงเทรา, จันทบุรี, ชลบุรี, ปราจีนบุรี, ระยอง, สระแก้ว, ตราด, นครนายก'
SW = ' กระบี่, พังงา, ภูเก็ต, ระนอง, สตูล, ตรัง'
SE = ' เพชรบุรี, ประจวบคีรีขันธ์, ประจวบคิรีขันธ์, ชุมพร, นครศรีธรรมราช, นราธิวาส, ปัตตานี, พัทลุง, สงขลา, สุราษฏร์ธานี, ยะลา'
Thai_sector = [NE, N, C, E, SE, SW]

In [135]:
Thai_dict = {f'{j}': ['จ.' + i.split(' ')[-1] if i.split(' ')[-1] != 'กทม.' else i.split(' ')[-1] for i in Thai_sector[j].split(',')] for j in range(len(Thai_sector))}

In [136]:
Thai_dict['4']

['จ.เพชรบุรี',
 'จ.ประจวบคีรีขันธ์',
 'จ.ประจวบคิรีขันธ์',
 'จ.ชุมพร',
 'จ.นครศรีธรรมราช',
 'จ.นราธิวาส',
 'จ.ปัตตานี',
 'จ.พัทลุง',
 'จ.สงขลา',
 'จ.สุราษฏร์ธานี',
 'จ.ยะลา']

In [137]:
swap_Thai_dict = {provide:sector for sector in Thai_dict.keys() for provide in Thai_dict[sector]}

In [138]:
swap_Thai_dict

{'จ.อำนาจเจริญ': '0',
 'จ.บึงกาฬ': '0',
 'จ.บุรีรัมย์': '0',
 'จ.ชัยภูมิ': '0',
 'จ.กาฬสินธุ์': '0',
 'จ.ขอนแก่น': '0',
 'จ.เลย': '0',
 'จ.มหาสารคาม': '0',
 'จ.มุกดาหาร': '0',
 'จ.นครพนม': '0',
 'จ.นครราชสีมา': '0',
 'จ.หนองบัวลำภู': '0',
 'จ.หนองคาย': '0',
 'จ.ร้อยเอ็ด': '0',
 'จ.สกลนคร': '0',
 'จ.ศรีสะเกษ': '0',
 'จ.สุรินทร์': '0',
 'จ.อุบลราชธานี': '0',
 'จ.อุดรธานี': '0',
 'จ.ยโสธร': '0',
 'จ.เชียงใหม่': '1',
 'จ.เชียงราย': '1',
 'จ.ลำปาง': '1',
 'จ.ลำพูน': '1',
 'จ.แม่ฮ่องสอน': '1',
 'จ.น่าน': '1',
 'จ.พะเยา': '1',
 'จ.แพร่': '1',
 'จ.อุตรดิตถ์': '1',
 'จ.ตาก': '1',
 'จ.สุโขทัย': '1',
 'จ.พิษณุโลก': '1',
 'จ.พิจิตร': '1',
 'จ.กำแพงเพชร': '1',
 'จ.เพชรบูรณ์': '1',
 'จ.นครสวรรค์': '2',
 'จ.อุทัยธานี': '2',
 'จ.อ่างทอง': '2',
 'จ.ชัยนาท': '2',
 'จ.พระนครศรีอยุธยา': '2',
 'กทม.': '2',
 'จ.ลพบุรี': '2',
 'จ.นครปฐม': '2',
 'จ.นนทบุรี': '2',
 'จ.ปทุมธานี': '2',
 'จ.สมุทรปราการ': '2',
 'จ.สมุทรสาคร': '2',
 'จ.สมุทรสงคราม': '2',
 'จ.สระบุรี': '2',
 'จ.สิงห์บุรี': '2',
 'จ.สุพรรณบุรี': '2',

In [144]:
def made_sector_station(selected_data_name, data_path, label_dict):
    df = pd.DataFrame(columns=['station_id', 'station_name'])
    for file_name in selected_data_name:
        station_detail = pd.read_excel(os.path.join(data_path, file_name), sheet_name='station_detail')
        select_detail = station_detail.copy()[['รหัสสถานี', 'ชื่อสถานี']].rename(columns={'รหัสสถานี':'station_id', 'ชื่อสถานี':'station_name'})
        df = pd.concat([df, select_detail], axis=0)
        df['station_name'] = df['station_name'].apply(lambda x: x.split(' ')[-1])
    df = df.drop_duplicates(subset=['station_id']).reset_index().drop(columns=['index'])
    df['label'] = df['station_name'].apply(lambda x: label_dict[x])
    return df

In [152]:
label_df = made_sector_station(selected_data_name, data_path, swap_Thai_dict)
label_df

,station_id,station_name,label
0,05T,กทม.,2
1,59T,กทม.,2
2,61T,กทม.,2
3,50T,กทม.,2
4,53T,กทม.,2
...,...,...,...
82,95T,จ.พิจิตร,1
83,96T,จ.สุโขทัย,1
84,97T,จ.เพชรบูรณ์,1
85,98T,จ.อุทัยธานี,2
